In [ ]:
import math
import warnings

from IPython.display import display
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy
import sklearn
from sklearn import preprocessing
from sklearn import linear_model
import statsmodels.formula.api as smf

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

# Non-DataScience/ML packages
import chardet
import re
import string

In [106]:
def find_suitable_variables():
    num_columns3 = num_columns
    cols3_orig = ['LarcenyTheft']
#     num_columns3.remove('Population')
    num_columns2 = ['PropertyCrime']
    max_r_squared = 0
    max_xy_pair = []
    max_xy_pair = []
    for Y_variable in num_columns2:
        if Y_variable != 'Population':
            Y = df[Y_variable].values.reshape(-1,1)
            print("Y variable is {}".format(Y_variable))
            for col in num_columns:
                cols3 = cols3_orig
                if col != Y_variable:
                    cols3.append(col)
                    print("Now calculating with X = {}, and Y_variable={}".format(cols3, Y_variable))
                    X = df[[cols3]]
                    regr.fit(X, Y)
                    # Inspect the results.
                    print('\nCoefficients: \n', regr.coef_)
                    print('\nIntercept: \n', regr.intercept_)
                    print('\nR-squared:')
                    rs = regr.score(X,Y)
#                     print("rs is a {} type variable".format(type(rs)))
#                     if max_r_squared == 0 or rs > max_r_squared:
#                         max_r_squared = rs.astype(type('float',(float,),{}))
#                         max_xy_pair[0] = Y_variable
#                         max_xy_pair[1] = col
                    print(regr.score(X, Y))
                    X = df[['Robbery','Burglary','Murder','Rape','LarcenyTheft','Arson']]
#     print("Golden Y variable={}, and X variable={}, and max_r_squared={}".format(max_xy_pair[0],
#                                                                              max_xy_pair[1],
#                                                                              max_r_squared))
#     return max_xy_pair, max_r_squared

In [26]:
path="../../../../Datafiles/"
file = 'table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls'
df = pd.read_excel(path + file, header=4)
print("there are {} entries in the data frame".format(len(df)))

there are 351 entries in the data frame


In [27]:
# Let's clean up the column names
colnames = df.columns
newcolnameslist = []

for colname in colnames:
    newcolname=colname.replace('\n','').replace(' ','').capitalize() # strip out newlines, spaces, and captitalize
    newcolname=re.sub(r'\([^)]*\)', '', newcolname)                  # remove parenthesized stuff
    newcolnameslist.append(newcolname)                               # put all the column names into a list 

df.columns = newcolnameslist
print(newcolnameslist)

['City', 'Population', 'Violentcrime', 'Murderandnonnegligentmanslaughter', 'Rape1', 'Rape2', 'Robbery', 'Aggravatedassault', 'Propertycrime', 'Burglary', 'Larceny-theft', 'Motorvehicletheft', 'Arson3']


In [29]:
df[df['City'].str.contains('figures shown')]
df[df['City'].str.contains('publish')]

df = df[(df['City'].str.contains('figures shown') == False)] # We don't want any documentation rows
df = df[(df['City'].str.contains('publish') == False)]       # We don't want any other documentation rows
df = df[(df['City'].str.contains('New York') == False)]       # Remove the outlier
df = df[(df['City'].str.contains('Buffalo') == False)]       # Remove the outlier
df = df[(df['City'].str.contains('Rochester') == False)]       # Remove the outlier
df = df[(df['City'].str.contains('Syracuse') == False)]       # Remove the outlier
df = df[(df['City'].str.contains('Yonkers') == False)]       # Remove the outlier

# All Rape1 values are false
# No Rape2 values are true
# Arson3 NaN's -> 161
# There are 161 NaN entries for Arson3
# Remove New York City -- it's an outlier.

In [30]:
df['PopulationSquared'] = df.Population.pow(2)

df['Robbery'] = np.where(df['Robbery']>=1, 1, 0.0)
df = df.rename(columns={'Arson3': 'Arson', 'Rape2': 'Rape', 'Aggravatedassault': 'AggravatedAssault',
                        'Violentcrime':'ViolentCrime','Propertycrime': 'PropertyCrime', 
                        'Larceny-theft':'LarcenyTheft','Motorvehicletheft':'MotorVehicleTheft'})
# df.Arson = df.Arson.astype(int) # Let's make it an int
df['Murder'] = np.where(df['Murderandnonnegligentmanslaughter']>=1, 1, 0.0)
df.Arson.fillna(0, inplace=True) # Let's clean up Arson3

In [31]:
df.head(10)
# df.describe()

,City,Population,ViolentCrime,Murderandnonnegligentmanslaughter,Rape1,Rape,Robbery,AggravatedAssault,PropertyCrime,Burglary,LarcenyTheft,MotorVehicleTheft,Arson,PopulationSquared,Murder
0,Adams Village,1861.000,0.000,0.000,nan,0.000,0.000,0.000,12.000,2.000,10.000,0.000,0.000,3463321.000,0.000
1,Addison Town and Village,2577.000,3.000,0.000,nan,0.000,0.000,3.000,24.000,3.000,20.000,1.000,0.000,6640929.000,0.000
2,Akron Village,2846.000,3.000,0.000,nan,0.000,0.000,3.000,16.000,1.000,15.000,0.000,0.000,8099716.000,0.000
3,Albany,97956.000,791.000,8.000,nan,30.000,1.000,526.000,4090.000,705.000,3243.000,142.000,0.000,9595377936.000,1.000
4,Albion Village,6388.000,23.000,0.000,nan,3.000,1.000,16.000,223.000,53.000,165.000,5.000,0.000,40806544.000,0.000
5,Alfred Village,4089.000,5.000,0.000,nan,0.000,1.000,2.000,46.000,10.000,36.000,0.000,0.000,16719921.000,0.000
6,Allegany Village,1781.000,3.000,0.000,nan,0.000,0.000,3.000,10.000,0.000,10.000,0.000,0.000,3171961.000,0.000
7,Amherst Town,118296.000,107.000,1.000,nan,7.000,1.000,68.000,2118.000,204.000,1882.000,32.000,3.000,13993943616.000,1.000
8,Amityville Village,9519.000,9.000,0.000,nan,2.000,1.000,3.000,210.000,16.000,188.000,6.000,1.000,90611361.000,0.000
9,Amsterdam,18182.000,30.000,0.000,nan,0.000,1.000,18.000,405.000,99.000,291.000,15.000,0.000,330585124.000,0.000


In [114]:
num_columns = ['Population', 'ViolentCrime', 'Rape', 'AggravatedAssault', 'PropertyCrime', 'Burglary','LarcenyTheft',
                                               'MotorVehicleTheft', 'Arson']

In [115]:
df_normalized.columns = num_columns
df_normalized.describe()

,Population,ViolentCrime,Rape,AggravatedAssault,PropertyCrime,Burglary,LarcenyTheft,MotorVehicleTheft,Arson
count,339.000,339.000,339.000,339.000,339.000,339.000,339.000,339.000,339.000
mean,0.113,0.038,0.056,0.035,0.074,0.064,0.075,0.055,0.020
std,0.150,0.109,0.132,0.101,0.124,0.127,0.124,0.132,0.081
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,0.021,0.003,0.000,0.002,0.010,0.008,0.009,0.000,0.000
50%,0.057,0.008,0.000,0.008,0.027,0.022,0.028,0.013,0.000
75%,0.148,0.027,0.065,0.026,0.082,0.065,0.085,0.042,0.000
max,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000


In [113]:
# Acquire, load, and preview the data.
regr = linear_model.LinearRegression()
     
Y = df['PropertyCrime'].values.reshape(-1,1)
X = df[['LarcenyTheft', 'MotorVehicleTheft']]
regr.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [[1.11540587 3.43737   ]]

Intercept: 
 [0.60164675]

R-squared:
0.9946028228964577


In [116]:
# work in process.
# find_suitable_variables()
# print("bigxy = {}, and max_r_squared={}".format(bigxy, max_r_squared))

My feature engineering included iterating through all of the numeric, non-PropertyCrime variables. I first came to using only LarcenyTheft, which yielded a 98% R-squared.  I thought that this was good, so then I iterated through the other possible variables, and found that the MotorVehicle theft addition, resulted in an R-squared of 99%, so I stopped there.